In [ ]:
!pip install vega altair

In [2]:
import pandas as pd
import altair as alt
alt.renderers.enable('default')

RendererRegistry.enable('default')

In [40]:
df = pd.read_csv('./Data.csv', encoding='latin-1')

In [41]:
df.head()

,date,articles_x_day,total_used_cloth(mts),initial_cloth,remaining_cloth,observations
0,1/01/2021,78,54.6,700.0,645.4,Cumple y pasa tope diario
1,2/01/2021,76,53.2,645.4,592.2,Cumple y pasa tope diario
2,3/01/2021,71,49.7,592.2,542.5,Casí logra meta diaria
3,4/01/2021,50,35.0,542.5,507.5,Falla maquina fileteadora
4,5/01/2021,72,50.4,507.5,457.1,Casí logra meta diaria


In [42]:
df.columns

Index(['date', 'articles_x_day', 'total_used_cloth(mts)', 'initial_cloth',
       'remaining_cloth', 'observations'],
      dtype='object')

In [43]:
print('Date: %d' % (df['date'].isnull().sum()))
print('Articles for Day: %d' % (df['articles_x_day'].isnull().sum()))

Date: 0
Articles for Day: 0


In [44]:
df.shape

(181, 6)

In [45]:
def extract_month(value):
    return pd.to_datetime(value, format = '%d/%m/%Y').month

In [46]:
# Extract month for each date field
df['monthNumber'] = df['date'].apply(extract_month)

In [47]:
df['month'] = df['monthNumber'].replace({
    1: 'Janaury',
    2: 'Febraury',
    3: 'March',
    4: 'April',
    5: 'May',
    6: 'June'
})

df.dtypes

date                      object
articles_x_day             int64
total_used_cloth(mts)    float64
initial_cloth            float64
remaining_cloth          float64
observations              object
monthNumber                int64
month                     object
dtype: object

In [48]:
sort_order = df.groupby(['monthNumber','month']).agg(['sum'])['articles_x_day']

In [49]:
bars = alt.Chart(df).mark_bar().encode(
    alt.X('sum(articles_x_day):Q'),
    alt.Y('monthNumber:O'),
    alt.Color('month:O'),
    tooltip = [alt.Tooltip('sum(articles_x_day):Q')]
).properties(
    width = 900,
    height = 200
)

text = alt.Chart(df).mark_text(dx=-30, dy=3, color='white').encode(
    x=alt.X('sum(articles_x_day):Q', stack='zero'),
    y=alt.Y('monthNumber:O'),
    detail='month:O',
    text=alt.Text('sum(articles_x_day):Q', format='.1f'))
    
bars + text

alt.LayerChart(...)

In [81]:
alt.Chart(df).mark_line().encode(
    alt.X('date:O'),
    alt.Y('articles_x_day:Q'),
    alt.Color('month:N')
).properties(
    width = 1000,
    height = 250
).interactive()

alt.Chart(...)

In [80]:
alt.Chart(df).mark_line().encode(
    alt.X('date:O'),
    alt.Y('total_used_cloth(mts):Q'),
    alt.Color('month:N')
).properties(
    width = 1000,
    height = 250
).interactive()

alt.Chart(...)

In [82]:
alt.Chart(df).mark_boxplot(size=50, extent=0.5).encode(
    alt.X('month:O'),
    alt.Y('sum(articles_x_day):Q',scale=alt.Scale(zero=False)),
    alt.Color('month:N')
).properties(
    width = 600,
    height = 250
)

alt.Chart(...)

In [52]:
df.sample()

,date,articles_x_day,total_used_cloth(mts),initial_cloth,remaining_cloth,observations,monthNumber,month
20,21/01/2021,71,49.7,399.7,350.0,Casí logra meta diaria,1,Janaury


In [64]:
cloth_resume_grouped_df = df.groupby('month').agg({
    'articles_x_day': 'mean',
    'initial_cloth': 'mean'
})\
.reset_index()\
.melt(id_vars = ['month'], var_name = 'Mes', value_name = 'Valor' )\
.sort_values(by = ['month', 'Mes']).round(2)

cloth_resume_grouped_df

,month,Mes,Valor
0,April,articles_x_day,73.90
6,April,initial_cloth,397.90
1,Febraury,articles_x_day,73.64
7,Febraury,initial_cloth,415.75
2,Janaury,articles_x_day,72.06
8,Janaury,initial_cloth,444.86
3,June,articles_x_day,72.33
9,June,initial_cloth,437.80
4,March,articles_x_day,73.29
10,March,initial_cloth,420.45


In [72]:
alt.Chart(cloth_resume_grouped_df).mark_line().encode(
    alt.X('month:O'),
    alt.Y('Valor:Q'),
    alt.Color('Mes:N')
).properties(
    width = 800,
    height = 300
)

alt.Chart(...)